In [1]:

from langchain.chains.conversation.memory import ConversationBufferWindowMemory,ConversationSummaryBufferMemory
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.agents.types import AgentType
from langchain.agents import initialize_agent
from langchain.tools import Tool
from langchain.prompts import MessagesPlaceholder
import requests
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.base import VectorStore
from langchain.vectorstores import Chroma
import chromadb
from langchain.prompts.chat import SystemMessagePromptTemplate

from dotenv import load_dotenv
import os
credential_path = f"../.credential"
# check is credential_path exist
assert os.path.exists(credential_path) is True
load_dotenv(credential_path)

def get_chromadb_setting():
    setting = chromadb.config.Settings()
    setting.persist_directory = f"../../vector_db/chroma"
    return setting


llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

# conversational_memory = ConversationBufferWindowMemory(
#     memory_key="chat_history", return_messages=True
# )
gpt_model = "gpt-3.5-turbo"
max_tokens = 500
conversational_memory = ConversationSummaryBufferMemory(
        llm=ChatOpenAI(temperature=0.0, max_tokens=max_tokens, model_name=gpt_model),
        # input_key="question",
        # output_key="answer",
        memory_key="chat_history",
        return_messages=True,
        max_token_limit=1000,
        # chat_memory=loaded_chat_memory,
        # chat_memory=[],
    )
def load_db_api(
    collection_name,
    embeddings=OpenAIEmbeddings(),
) -> VectorStore:
    db = None
    # try:
    #     url = f"http://localhost:8000/api/v1/collections/{collection_name}"
    #     response = requests.get(url)
    #     assert response.status_code == 200
    #     db = Chroma(
    #         collection_name=collection_name,
    #         client=chromadb.HttpClient(),
    #         embedding_function=embeddings,
    #     )

    #     print("\tLoad chroma index success")
    # except Exception:
    #     raise LoadDBException("Load chroma index failed")
    # url = f"http://localhost:8000/api/v1/collections/{collection_name}"
    print("\n\there,")
    try:
        url = f"http://localhost:8000/api/v1/collections/{collection_name}"
        response = requests.get(url)
        print("\n\there,", response.status_code, url)
    except:
        pass
    # try:
    #     url = f"http://chroma_server:8000/api/v1/collections/{collection_name}"
    #     response = requests.get(url)
    #     print("\n\there,", response.status_code, url)
    # except:
    #     pass
    # try:
    #     url = f"http://server:8000/api/v1/collections/{collection_name}"
    #     response = requests.get(url)
    #     print("\n\there,", response.status_code, url)
    # except:
    #     pass
    assert response.status_code == 200

    db = Chroma(
        collection_name=collection_name,
        client=chromadb.HttpClient(
            host="localhost", settings=get_chromadb_setting()
        ),
        embedding_function=embeddings,
    )

    print("\tLoad chroma index success")
    return db




In [2]:
db = load_db_api("product_a_small")


	here,

	here, 200 http://localhost:8000/api/v1/collections/product_a_small
	Load chroma index success


In [3]:
from langchain.schema import SystemMessage
from langchain.agents import OpenAIFunctionsAgent
system_message = SystemMessage(content="You are very powerful assistant, but bad at calculating lengths of words.")


In [4]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=MEMORY_KEY)]
)

In [5]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

In [6]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=MEMORY_KEY)]
)

In [7]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)
llm = ChatOpenAI(temperature=0)



In [8]:
from langchain.agents import AgentExecutor
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)


agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)
agent_executor.run("how many letters in the word educa?")
agent_executor.run("is that a real word?")



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


> Entering new AgentExecutor chain...
No, "educa" is not a real word in English.

> Finished chain.


'No, "educa" is not a real word in English.'

In [10]:
type(agent_executor)

langchain.agents.agent.AgentExecutor

In [9]:
raise

RuntimeError: No active exception to reraise

In [ ]:

from langchain import PromptTemplate, LLMChain
class NoOpLLMChain(LLMChain):
    """No-op LLM chain."""

    def __init__(self):
        """Initialize."""
        super().__init__(
            llm=ChatOpenAI(), prompt=PromptTemplate(template="", input_variables=[])
        )

    def run(self, question: str, *args, **kwargs) -> str:
        return question


In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryBufferMemory, ChatMessageHistory

memory = ConversationSummaryBufferMemory(
        llm=ChatOpenAI(temperature=0.0, max_tokens=max_tokens, model_name=gpt_model),
        input_key="question",
        output_key="answer",
        memory_key="chat_history",
        return_messages=True,
        max_token_limit=1000,
        chat_memory=ChatMessageHistory(),
    )
other_qa = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(temperature=0.0, max_tokens=max_tokens, model_name=gpt_model),
    chain_type="stuff",
    memory=memory,
    retriever=db.as_retriever(),
    return_generated_question=True,
    return_source_documents=True,
)

no_op_chain = NoOpLLMChain()
other_qa.question_generator = no_op_chain
modified_template = "Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}\nChat History:\n{chat_history}"
system_message_prompt = SystemMessagePromptTemplate.from_template(modified_template)
other_qa.combine_docs_chain.llm_chain.prompt.messages[0] = system_message_prompt

# add chat_history as a variable to the llm_chain's ChatPromptTemplate object
other_qa.combine_docs_chain.llm_chain.prompt.input_variables = [
    "context",
    "question",
    "chat_history",
]


In [ ]:
other_qa.combine_docs_chain.llm_chain.__dict__

{'memory': None,
 'callbacks': None,
 'callback_manager': None,
 'verbose': False,
 'tags': None,
 'metadata': None,
 'prompt': ChatPromptTemplate(input_variables=['context', 'question', 'chat_history'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['chat_history', 'context'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}\nChat History:\n{chat_history}", template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='{question}', template_format='f-string', validate_template=True), additional_kwargs={})]),
 'llm': ChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manage

In [ ]:
from langchain.memory import ConversationBufferMemory,ConversationTokenBufferMemory

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
)
# memory = ConversationSummaryBufferMemory(
#         llm=ChatOpenAI(temperature=0.0, max_tokens=max_tokens, model_name=gpt_model),
#         input_key="question",
#         output_key="answer",
#         memory_key="chat_history",
#         return_messages=True,
#         max_token_limit=1000,
#         chat_memory=ChatMessageHistory(),
#     )
# qa = ConversationalRetrievalChain.from_llm(
#     llm=ChatOpenAI(temperature=0.0, max_tokens=max_tokens, model_name=gpt_model),
#     chain_type="stuff",
#     memory=memory,
#     retriever=db.as_retriever(),
#     return_generated_question=True,
#     return_source_documents=True,
# )

# no_op_chain = NoOpLLMChain()
# qa.question_generator = no_op_chain
# modified_template = "Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}\nChat History:\n{chat_history}"
# system_message_prompt = SystemMessagePromptTemplate.from_template(modified_template)
# qa.combine_docs_chain.llm_chain.prompt.messages[0] = system_message_prompt

# # add chat_history as a variable to the llm_chain's ChatPromptTemplate object
# qa.combine_docs_chain.llm_chain.prompt.input_variables = [
#     "context",
#     "question",
#     "chat_history",
# ]

# system_message = """
#         You are a Product information chatbot name naxon( first letter n from neuron and axon from axon). 
#         You can ask questions to help you understand user intent.
#         You should only talk within the context of problem.
#         If you are unsure of how to help, you can suggest the client to contact the customer support.
#         You should talk on Thai, unless the client talks in English. """
system_message = """
        You are a Product information chatbot name naxon( first letter n from neuron and axon from axon). 
        You can ask questions to help you understand user intent.
        You should only talk within the context of problem.
        If you are unsure of how to help, you can suggest the client to contact the customer support.
        You should talk on Thai, unless the client talks in English. 
        ####
        Chat History:\n{chat_history}"
        """
# modified_template = "Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}\nChat History:\n{chat_history}"
# system_message_prompt = SystemMessagePromptTemplate.from_template(modified_template)

# qa.combine_documents_chain.llm_chain.prompt.messages[0] = modified_template
# qa.combine_documents_chain.llm_chain.prompt.input_variables = [
#         "context",
#         "question",
#         "chat_history",
#     ]
tools = [
    Tool(
        name="qa-product-information",
        func=qa.run,
        description="Useful when you need to information about the product.",
    )
]
MEMORY_KEY = "chat_history"
memory = ConversationTokenBufferMemory(llm=llm,memory_key=MEMORY_KEY,max_token_limit=1000, return_messages=True)

executor = initialize_agent(
    agent = AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    memory=memory,
    # memory=conversational_memory,
    max_iterations=3,
    agent_kwargs={"system_message": system_message},
    verbose=True,
)


In [ ]:
executor.__dict__['agent'].__dict__['llm_chain'].__dict__.keys()


dict_keys(['memory', 'callbacks', 'callback_manager', 'verbose', 'tags', 'metadata', 'prompt', 'llm', 'output_key', 'output_parser', 'return_final_only', 'llm_kwargs'])

In [ ]:
other_qa_dict = other_qa.combine_docs_chain.llm_chain.__dict__
qa_dict = executor.__dict__['agent'].__dict__['llm_chain'].__dict__

for key in other_qa_dict.keys():
    print("\t",key)
    if other_qa_dict[key] != qa_dict[key]:
        print("other_qa_dict\t",other_qa_dict[key])
        print("qa_dict\t\t",qa_dict[key])

        print()
for key in other_qa_dict.keys():
    if key !="prompt":
        continue
    qa_prompt = qa_dict[key]
    other_qa_prompt = other_qa_dict[key]

	 memory
	 callbacks
	 callback_manager
	 verbose
	 tags
	 metadata
	 prompt
other_qa_dict	 input_variables=['context', 'question', 'chat_history'] output_parser=None partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['chat_history', 'context'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}\nChat History:\n{chat_history}", template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='{question}', template_format='f-string', validate_template=True), additional_kwargs={})]
qa_dict		 input_variables=['input', 'chat_history', 'agent_scratchpad'] output_parser=None partial_variables={} messages=[SystemMessagePr

In [ ]:
qa_prompt.__dict__

{'input_variables': ['input', 'chat_history', 'agent_scratchpad'],
 'output_parser': None,
 'partial_variables': {},
 'messages': [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['chat_history'], output_parser=None, partial_variables={}, template='\n        You are a Product information chatbot name naxon( first letter n from neuron and axon from axon). \n        You can ask questions to help you understand user intent.\n        You should only talk within the context of problem.\n        If you are unsure of how to help, you can suggest the client to contact the customer support.\n        You should talk on Thai, unless the client talks in English. \n        ####\n        Chat History:\n{chat_history}"\n        ', template_format='f-string', validate_template=True), additional_kwargs={}),
  MessagesPlaceholder(variable_name='chat_history'),
  HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, templat

In [ ]:
other_qa_prompt.__dict__

{'input_variables': ['context', 'question', 'chat_history'],
 'output_parser': None,
 'partial_variables': {},
 'messages': [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['chat_history', 'context'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}\nChat History:\n{chat_history}", template_format='f-string', validate_template=True), additional_kwargs={}),
  HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='{question}', template_format='f-string', validate_template=True), additional_kwargs={})]}

In [ ]:
from langchain.prompts import MessagesPlaceholder
from langchain.agents import OpenAIFunctionsAgent
from langchain.agents import AgentExecutor
from langchain.schema import SystemMessage
from langchain.agents import OpenAIFunctionsAgent
system_message = SystemMessage(content="You are very powerful assistant, but bad at calculating lengths of words.")
MEMORY_KEY = "chat_history"
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=MEMORY_KEY)]
)

In [ ]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

In [ ]:
from langchain.memory import ConversationBufferMemory,ConversationTokenBufferMemory
# memory = ConversationTokenBufferMemory(llm=llm,memory_key=MEMORY_KEY,max_token_limit=1000, return_messages=True)
memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)

In [ ]:
from langchain.agents import ConversationalChatAgent
from langchain.agents import AgentExecutor


# agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
agent = ConversationalChatAgent.from_llm_and_tools(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)
agent_executor.run("how many letters in the word educa?")
agent_executor.run("is that a real word?")



> Entering new AgentExecutor chain...
{
    "action": "get_word_length",
    "action_input": "educa"
}
Observation: 5
Thought:{
    "action": "Final Answer",
    "action_input": "5"
}

> Finished chain.


> Entering new AgentExecutor chain...
{
    "action": "get_word_length",
    "action_input": "educa"
}
Observation: 5
Thought:{
    "action": "Final Answer",
    "action_input": "Yes, 'educa' is a real word."
}

> Finished chain.


"Yes, 'educa' is a real word."

In [ ]:
system_message_content = """You are a Product information chatbot name naxon( first letter n from neuron and axon from axon). 
        You can ask questions to help you understand user intent.
        You should only talk within the context of problem.
        If you are unsure of how to help, you can suggest the client to contact the customer support.
        You should talk on Thai, unless the client talks in English. 
        """
system_message = SystemMessage(content=system_message_content)
MEMORY_KEY = "chat_history"
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=MEMORY_KEY)]
)
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], output_parser=None, partial_variables={}, messages=[SystemMessage(content='You are a Product information chatbot name naxon( first letter n from neuron and axon from axon). \n        You can ask questions to help you understand user intent.\n        You should only talk within the context of problem.\n        If you are unsure of how to help, you can suggest the client to contact the customer support.\n        You should talk on Thai, unless the client talks in English. \n        ', additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='{input}', template_format='f-string', validate_template=True), additional_kwargs={}), MessagesPlaceholder(variable_name='agent_scratchpad')])

In [ ]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from typing import Optional, Type
from pydantic import BaseModel, Field
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)


class CalculatorInput(BaseModel):
    question: str = Field()

class CustomCalculatorTool(BaseTool):
    name = "Calculator"
    description = "useful for when you need to answer questions about math"
    args_schema: Type[BaseModel] = CalculatorInput

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return llm_math_chain.run(query)


In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
)

tools = [
    Tool(
        name="qa-product-information",
        func=qa.run,
        description="Useful when you need to information about the product.",
    ).args("chat_history")
]
# agent = ConversationalChatAgent.from_llm_and_tools(llm=llm, tools=tools, prompt=prompt)
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)

executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)


TypeError: 'dict' object is not callable

In [ ]:
prompt = "what is the dimension of washing machine?"
executor.run(prompt)
# executor.run({"input":prompt,"chat_history":chat_history})



> Entering new AgentExecutor chain...

Invoking: `qa-product-information` with `washing machine`




ValueError: Missing some input keys: {'chat_history'}

In [ ]:
executor.memory.__dict__['chat_memory'].__dict__

{'messages': []}

In [ ]:
raise

In [ ]:
prompt = "what did I just say?"
executor.run(prompt)



> Entering new AgentExecutor chain...
{
    "action": "qa-product-information",
    "action_input": "what did I just say?"
}
Observation: You did not say anything in the provided context.
Thought:{
    "action": "Final Answer",
    "action_input": "I apologize, but I am not able to remember or retrieve previous tool responses. Is there anything else I can assist you with?"
}

> Finished chain.


'I apologize, but I am not able to remember or retrieve previous tool responses. Is there anything else I can assist you with?'

In [ ]:
executor.memory.__dict__['chat_memory'].__dict__

{'messages': [HumanMessage(content='what is the dimension of washing machine?', additional_kwargs={}, example=False),
  AIMessage(content='The dimensions of the washing machine are 848 x 596.5 x 575 mm (for models EWF8024P5WB and EWF8024P5SB) and 848 x 596.5 x 659 mm (for model EWF9024P5WB).', additional_kwargs={}, example=False),
  HumanMessage(content='what is the dimension of washing machine?', additional_kwargs={}, example=False),
  AIMessage(content='The dimensions of the washing machine are 848 x 596.5 x 575 mm (for models EWF8024P5WB and EWF8024P5SB) and 848 x 596.5 x 659 mm (for model EWF9024P5WB).', additional_kwargs={}, example=False),
  HumanMessage(content='what did I just say?', additional_kwargs={}, example=False),
  AIMessage(content='I apologize, but I am not able to remember or retrieve previous tool responses. Is there anything else I can assist you with?', additional_kwargs={}, example=False)]}

In [ ]:
executor.__dict__['agent'].__dict__['llm_chain'].__dict__['prompt'].__dict__['messages'][0].__dict__

{'prompt': PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='\n        You are a Product information chatbot name naxon( first letter n from neuron and axon from axon). \n        You can ask questions to help you understand user intent.\n        You should only talk within the context of problem.\n        If you are unsure of how to help, you can suggest the client to contact the customer support.\n        You should talk on Thai, unless the client talks in English.\n        ', template_format='f-string', validate_template=True),
 'additional_kwargs': {}}

In [ ]:
prompt = "สวัสดี"
try:
    response= executor.run(prompt)
except Exception as e:
        response = str(e)
        if response.startswith("Could not parse LLM output:"):
            response = response.removeprefix("Could not parse LLM output:").removesuffix("`")
response



> Entering new AgentExecutor chain...


' สวัสดีครับ! ฉันคือ Naxon ช่วยเหลือคุณได้อย่างไรบ้างครับ?'

In [ ]:
prompt = "hi my name is ford"
try:
    response= executor.run(prompt)
except Exception as e:
        response = str(e)
        if response.startswith("Could not parse LLM output:"):
            response = response.removeprefix("Could not parse LLM output:").removesuffix("`")
response



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "Hi Ford! How can I assist you today?"
}

> Finished chain.


'Hi Ford! How can I assist you today?'

In [ ]:
prompt = "what is my name?"
try:
    response= executor.run(prompt)
except Exception as e:
        response = str(e)
        if response.startswith("Could not parse LLM output:"):
            response = response.removeprefix("Could not parse LLM output:").removesuffix("`")
response



> Entering new AgentExecutor chain...


" I'm sorry, but as an AI chatbot, I don't have access to personal information. Therefore, I don't know your name. Is there anything else I can assist you with?"